# 0 Importando dependencias 

In [1]:
!pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]

In [2]:
import gym 
import ale_py
import random

C:\Users\migue\AppData\Local\Programs\Python\Python38\lib\site-packages\ale_py\gym\environment.py:11: DeprecationWarning: Importing atari-py roms won't be supported in future releases of ale-py.
  import ale_py.roms as roms
C:\Users\migue\AppData\Local\Programs\Python\Python38\lib\site-packages\ale_py\roms\__init__.py:44: UserWarning: atari_py.atari_roms contains unsupported ROMs: C:\Users\migue\AppData\Local\Programs\Python\Python38\Lib\site-packages\atari_py\atari_roms\{adventure.bin, air_raid.bin, alien.bin, amidar.bin, assault.bin, asterix.bin, asteroids.bin, atlantis.bin, bank_heist.bin, battle_zone.bin, beam_rider.bin, berzerk.bin, bowling.bin, boxing.bin, breakout.bin, carnival.bin, centipede.bin, chopper_command.bin, crazy_climber.bin, defender.bin, demon_attack.bin, double_dunk.bin, elevator_action.bin, enduro.bin, fishing_derby.bin, freeway.bin, frostbite.bin, gopher.bin, gravitar.bin, hero.bin, ice_hockey.bin, jamesbond.bin, journey_escape.bin, kaboom.bin, kangaroo.bin, krul

# 1 Teste aleatorio utilizando OpenAI Gym

In [3]:
env = gym.make('Assault-v4')# O jogo escolhido para o aprendizado de maquina foi o  do console Atari
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [4]:
env.unwrapped.get_action_meanings() # todos os comandos do jogo listados

['NOOP', 'FIRE', 'UP', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [6]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5,6])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:210.0
Episode:2 Score:252.0
Episode:3 Score:378.0
Episode:4 Score:231.0
Episode:5 Score:210.0


# 2 Criando Modelo de Deep Learning Keras

In [7]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [8]:
def build_model(height, width, channels, actions):
    model = Sequential() #Adicionando camadas de Convolução por pixels
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten()) #Juntando as camadas geradas para que estajam totalmente acopladas
    model.add(Dense(512, activation='relu')) #camadas de densidade
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [15]:
del model

In [16]:
model = build_model(height, width, channels, actions)

In [17]:
model.summary() #Mostrando o sumario com a relação de todas as camadas e variaveis de treinamento

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 61, 39, 32)     6176      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 29, 18, 64)     32832     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 27, 16, 64)     36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 82944)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               42467840  
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_6 (Dense)              (None, 7)                

# 3 Criando a entidade que será treinada com Keras-RL

In [18]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [19]:
def build_agent(model, actions): #Criando o metodo de aprendizagem da maquina, eps greedy
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3) #memoria do agente, sequencial
    dqn = DQNAgent(model=model, memory=memory, policy=policy, #Network de reforço
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [20]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4)) # Realizando o Treinamento

In [20]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 10000 steps ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
  519/10000: episode: 1, duration: 19.061s, episode steps: 519, steps per second:  27, episode reward: 210.000, mean reward:  0.405 [ 0.000, 21.000], mean action: 3.073 [0.000, 6.000],  loss: --, mean_q: --, mean_eps: --
 1206/10000: episode: 2, duration: 320.754s, episode steps: 687, steps per second:   2, episode reward: 189.000, mean reward:  0.275 [ 0.000, 21.000], mean action: 3.093 [0.000, 6.000],  loss: 20.274902, mean_q: 10.399663, mean_eps: 0.900730
 1884/10000: episode: 3, duration: 981.156s, episode steps: 678, steps per second:   1, episode reward: 189.000, mean reward:  0.279 [ 0.000, 21.000], mean action: 2.917 [0.000, 6.000],  loss: 0.910981, mean_q: 9.292033, mean_eps: 0.860995
 2646/10000: episode: 4, duration: 1104.903s, episode steps: 762, steps per second:   1, episode reward: 189.000, mean reward:  0.248 [ 0.000, 21.000],

In [21]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward'])) # Testando a IA apos o treinamento

Testing for 10 episodes ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Episode 1: reward: 441.000, steps: 937
Episode 2: reward: 168.000, steps: 703
Episode 3: reward: 273.000, steps: 988
Episode 4: reward: 273.000, steps: 812
Episode 5: reward: 147.000, steps: 557
Episode 6: reward: 168.000, steps: 478
Episode 7: reward: 378.000, steps: 877
Episode 8: reward: 210.000, steps: 696
Episode 9: reward: 357.000, steps: 947
Episode 10: reward: 315.000, steps: 825
273.0


# 4. Recuperenado IA treinada da memoria

In [21]:
dqn.save_weights('SavedWeights/10k-Fast/dqn_weights.h5f')

[WARNING] SavedWeights/10k-Fast/dqn_weights.h5f.index already exists - overwrite? [y/n]y
[TIP] Next time specify overwrite=True!


In [31]:
del model, dqn

NameError: name 'model' is not defined

In [19]:
dqn.load_weights('SavedWeights/10k-Fast/dqn_weights.h5f')